# Color Distance

Find pairs of colors, from  measured values form the color sensor, that are maximally far apart. 

In [5]:
# These are the colors in LAB space, as a dictionary with color names as keys.
learnedColors = {
    'Yellow': [52, -1, 16],
    'Green': [55, -11, 23],
    'Blue': [57, -3, 2],
    'Purple': [54, 0, 11],
    'Red': [54, 3, 12],
    'Orange': [53, 6, 15],
    'Pink': [53, 7, 13]
}


In [6]:
import numpy as np

# Convert the color dictionary to a list of names and a numpy array of LAB values
color_names = list(learnedColors.keys())
color_values = np.array(list(learnedColors.values()))

# Compute pairwise Euclidean distances
distances = np.linalg.norm(color_values[:, np.newaxis, :] - color_values[np.newaxis, :, :], axis=2)

# For each color, find the index of the farthest color
farthest = {}
for i, name in enumerate(color_names):
    # Exclude self (distance zero)
    farthest_idx = np.argmax(distances[i] + np.eye(len(color_names))[i]*-np.inf)
    farthest[name] = color_names[farthest_idx]

farthest

/var/folders/ft/vb_vz6jn48s5nr_fr9mhxt8m0000gn/T/ipykernel_64538/2181827254.py:14: RuntimeWarning: invalid value encountered in multiply
  farthest_idx = np.argmax(distances[i] + np.eye(len(color_names))[i]*-np.inf)


{'Yellow': 'Green',
 'Green': 'Yellow',
 'Blue': 'Yellow',
 'Purple': 'Yellow',
 'Red': 'Yellow',
 'Orange': 'Yellow',
 'Pink': 'Yellow'}

In [7]:
# Compute the distance matrix (already computed as 'distances')
# Get unique distances (excluding zero/self-distances), sorted
unique_distances = np.unique(distances[np.triu_indices(len(color_names), k=1)])
unique_distances_sorted = np.sort(unique_distances)
print("Unique sorted distances:", unique_distances_sorted)

# Greedy selection of farthest pairs, removing both colors from the pool after each selection
remaining_indices = set(range(len(color_names)))
pairs = []
while len(remaining_indices) > 1:
    # Find the pair with the largest distance among remaining colors
    max_dist = -np.inf
    max_pair = None
    for i in remaining_indices:
        for j in remaining_indices:
            if i < j and distances[i, j] > max_dist:
                max_dist = distances[i, j]
                max_pair = (i, j)
    if max_pair is None:
        break
    i, j = max_pair
    pairs.append(((color_names[i], color_names[j]), max_dist))
    # Remove both colors from the pool
    remaining_indices.remove(i)
    remaining_indices.remove(j)

print("Farthest pairs (greedy, removing both colors each time):")
for (c1, c2), dist in pairs:
    print(f"{c1} - {c2}: {dist:.2f}")

Unique sorted distances: [ 2.23606798  3.16227766  4.24264069  4.35889894  5.47722558  6.
  7.14142843  7.28010989  7.34846923  8.60232527  9.94987437 12.04159458
 12.56980509 15.         15.39480432 16.30950643 17.8325545  18.89444363
 20.68816087 22.56102835]
Farthest pairs (greedy, removing both colors each time):
Green - Blue: 22.56
Yellow - Pink: 8.60
Purple - Orange: 7.28


In [8]:
import itertools

# Exclude one color to get 6 colors
six_color_indices = [i for i in range(len(color_names)) if i != 4]  # Exclude 'Red' (index 4) for example
six_color_names = [color_names[i] for i in six_color_indices]

# All combinations of 3 out of 6
best_groups = None
best_mean_rms = -np.inf

for group1_indices in itertools.combinations(range(6), 3):
    group2_indices = [i for i in range(6) if i not in group1_indices]
    # Get indices in the original color_values array
    group1 = [six_color_indices[i] for i in group1_indices]
    group2 = [six_color_indices[i] for i in group2_indices]
    # Compute pairwise distances for each group
    dists1 = []
    for i, j in itertools.combinations(group1, 2):
        dists1.append(distances[i, j])
    dists2 = []
    for i, j in itertools.combinations(group2, 2):
        dists2.append(distances[i, j])
    rms1 = np.sqrt(np.mean(np.square(dists1)))
    rms2 = np.sqrt(np.mean(np.square(dists2)))
    mean_rms = (rms1 + rms2) / 2
    if mean_rms > best_mean_rms:
        best_mean_rms = mean_rms
        best_groups = ([color_names[i] for i in group1], [color_names[i] for i in group2])

print("Best grouping (max mean RMS distance):")
print("Group 1:", best_groups[0])
print("Group 2:", best_groups[1])
print("Mean RMS distance:", best_mean_rms)

Best grouping (max mean RMS distance):
Group 1: ['Yellow', 'Blue', 'Orange']
Group 2: ['Green', 'Purple', 'Pink']
Mean RMS distance: 14.615761239436658
